<h1 style="color:#2E86C1; font-family: 'Helvetica';">srf.py</h2>

<p style="font-size: 16px; line-height: 1.5; color:#333;">
This is the <b>Gaussian Random Field</b> generator to create spatial correlated cost fields.
Implementation is done using different available kernels that are supported by <b>Fast Fourier Transformation</b> in <b>GSTools</b>. Import parameters are passed on by <i>parameters.json</i> to enable batch export for high-volume testing.
</p>

<h3 style="color:#2E86C1; font-family: 'Helvetica';">==Package imports==</h2>

<p style="font-size: 16px; line-height: 1.5; color:#333;">
The data preprocessing part:</p>

<p>- os </p>
<p>- json </p>
<p>- itertools </p>
<p>- matplotlib </p>
<p>- gstools </p>
<p>- pandas </p>

In [577]:
import os, json, itertools
import numpy as np
import matplotlib.pyplot as plt
import gstools as gs
from gstools.field import SRF
from gstools.field.generator import Fourier
from gstools.random import MasterRNG
from gstools.covmodel import Gaussian, Matern, Exponential, Stable, Rational
from datetime import datetime
import pandas as pd

<h3 style="color:#2E86C1; font-family: 'Helvetica';">(1) Data Preprocessing</h2>

<p style="font-size: 16px; line-height: 1.5; color:#333;">
The data preprocessing part:</p>

In [578]:
def make_dir(path):
    os.makedirs(path, exist_ok=True)
    return path

In [579]:
def build_nodes(grid_size):
    nodes = pd.DataFrame({
        "node_id":  np.arange(grid_size*grid_size),
        "x":        np.repeat(np.arange(grid_size), grid_size),
        "y":        np.tile(np.arange(grid_size), grid_size)
    })
    return nodes

In [580]:
def build_edges_torus(grid_size: int) -> pd.DataFrame:
    edges = []
    edge_id = 0
    N = grid_size

    def node_id(i, j):
        return i * N + j

    for i in range(N):
        for j in range(N):
            u = node_id(i, j)

            # Right neighbor (wrap horizontally)
            v_right = node_id(i, (j + 1) % N)
            edges.append((edge_id, u, v_right))
            edge_id += 1

            # Up neighbor (wrap vertically)
            v_up = node_id((i + 1) % N, j)
            edges.append((edge_id, u, v_up))
            edge_id += 1
    
    edges_df = pd.DataFrame(edges, columns=["edge_id", "u", "v"])

    return edges_df

In [581]:
def build_edges_4conn_nonperiodic(grid_size, nominal_field, deviation_field=None):
    """
    Builds a 4-connected *non-periodic* grid graph.
    Each node connects to up to 4 neighbors (no wrap-around).
    """
    import pandas as pd
    N = grid_size

    def nid(i, j):  # convert grid coordinate to node ID
        return i * N + j

    rows = []
    edge_id = 0
    for i in range(N):
        for j in range(N):
            u = nid(i, j)

            # 4 neighbors WITHOUT wrap-around
            nbrs = []
            if j + 1 < N:  # right
                nbrs.append((i, j + 1))
            if j - 1 >= 0:  # left
                nbrs.append((i, j - 1))
            if i + 1 < N:  # down
                nbrs.append((i + 1, j))
            if i - 1 >= 0:  # up
                nbrs.append((i - 1, j))

            for (ni, nj) in nbrs:
                v = nid(ni, nj)
                nominal = float(nominal_field[i, j])
                deviation = float(deviation_field[i, j]) if deviation_field is not None else 0.0
                cost = nominal + deviation
                rows.append({
                    "edge_id": edge_id,
                    "u": u, "v": v,
                    "nominal": nominal,
                    "deviation": deviation,
                    "cost": cost
                })
                edge_id += 1

    return pd.DataFrame(rows)

In [582]:
def build_edges(grid_size):
    edges = []
    edge_id = 0
    for x in range(grid_size):
        for y in range(grid_size):
            node_index = x * grid_size + y
            # Right neighbor
            if y + 1 < grid_size:
                edges.append((edge_id, node_index, node_index + 1))
                edge_id += 1
            # Bottom neighbor
            if x + 1 < grid_size:
                edges.append((edge_id, node_index, node_index + grid_size))
                edge_id += 1
    edges_df = pd.DataFrame(edges, columns=["edge_id", "u", "v"])
    return edges_df

In [583]:
def attach_costs(edges_df, field):
    costs = []
    for _, row in edges_df.iterrows():
        cost = (field.flat[row.u] + field.flat[row.v]) / 2
        costs.append(cost)
    edges_df = edges_df.copy()
    edges_df["cost"] = costs
    return edges_df

<h3 style="color:#2E86C1; font-family: 'Helvetica';">(2) Data PostProcessing Functions</h2>

<p style="font-size: 16px; line-height: 1.5; color:#333;">
Functions that are used in data postprocessing:</p>

In [584]:
def normalize_field(field):
    field -= np.nanmin(field)
    max_val = np.nanmax(field)
    if max_val > 0:
        field /= max_val
    return field

In [585]:
def rescale_field(field, low=1, high=100):
    field = normalize_field(field)
    return low + (high - low) * field

In [586]:
def save_plot(field, title, path):
    plt.figure()
    plt.imshow(field.T, origin="lower")
    plt.colorbar(label="cost")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(path, dpi=150)
    plt.close()

<h3 style="color:#2E86C1; font-family: 'Helvetica';">(3) SRF Generation</h2>

<p style="font-size: 16px; line-height: 1.5; color:#333;">
The data preprocessing part:</p>

In [587]:
def generate_grf(kernel_name, dim, var, len_scale, nugget, nu, alpha, beta, period, seed):
    if kernel_name == "Gaussian":
        actual_model   = Gaussian(dim=dim, var=var, len_scale=len_scale)
        forecast_model = Gaussian(dim=dim, var=var, len_scale=len_scale, nugget=nugget)
    elif kernel_name == "Matern":
        actual_model   = Matern(dim=dim, var=var, len_scale=len_scale, nu=nu)
        forecast_model = Matern(dim=dim, var=var, len_scale=len_scale, nu=nu, nugget=nugget)
    elif kernel_name == "Exponential":
        actual_model   = Exponential(dim=dim, var=var, len_scale=len_scale)
        forecast_model = Exponential(dim=dim, var=var, len_scale=len_scale, nugget=nugget)
    elif kernel_name == "Stable":
        actual_model   = Stable(dim=dim, var=var, len_scale=len_scale, alpha=alpha)
        forecast_model = Stable(dim=dim, var=var, len_scale=len_scale, alpha=alpha, nugget=nugget)
    elif kernel_name == "Rational":
        actual_model   = Rational(dim=dim, var=var, len_scale=len_scale, beta=beta)
        forecast_model = Rational(dim=dim, var=var, len_scale=len_scale, beta=beta, nugget=nugget)
    else:
        raise ValueError(f"Unknown kernel: {kernel_name}")

    actual_srf     = SRF(actual_model, generator=Fourier, seed=seed, period=period)
    forecast_srf   = SRF(forecast_model, generator=Fourier, seed=seed, period=period)

    return actual_srf, forecast_srf

<h3 style="color:#2E86C1; font-family: 'Helvetica';">(4) Data Postprocessing</h2>

<p style="font-size: 16px; line-height: 1.5; color:#333;">
The data preprocessing part:</p>

In [588]:
def DataPostProcessingSRF(param_file="parameters.json"):
    with open(param_file, "r") as f:
        params = json.load(f)

    general_params = params["general"]
    field_params   = params["field"]
    grid_params    = params["grid"]

    # Prepare RNG
    #masterseed  = general_params['master_seed']
    #master      = MasterRNG(masterseed)
    #seeds       = [master() for _ in range(general_params["num_seeds"])]

    # === Multi-master seed support (robust to both keys/types) ===
    raw_master = general_params.get("master_seeds",
                                    general_params.get("master_seed", 11))

    # Normalize to a flat list of ints
    if isinstance(raw_master, int):
        master_seed_list = [raw_master]
    elif isinstance(raw_master, list):
        master_seed_list = raw_master
    else:
        raise ValueError("general.master_seed(s) must be int or list[int].")

    num_seeds_per_master = general_params.get("num_seeds_per_master",
                                            general_params.get("num_seeds", 1))

    all_seed_combos = []
    for master_seed in master_seed_list:
        master = MasterRNG(int(master_seed))
        sub_seeds = [int(master()) for _ in range(int(num_seeds_per_master))]
        all_seed_combos.extend([(int(master_seed), s) for s in sub_seeds])

    print(f"→ Running {len(all_seed_combos)} total scenarios from "
        f"{len(master_seed_list)} master seeds: {master_seed_list}")



    out_dir = make_dir(general_params["output_dir"])

    # Cartesian product of parameters
    for master_seed, seed in all_seed_combos:
        for grid_size in grid_params["sizes"]:
            # Create grid
            x       = np.linspace(0, grid_size, grid_size, endpoint=False)
            y       = np.linspace(0, grid_size, grid_size, endpoint=False)
            grid    = (x, y)
            period  = grid_size

            for kernel_name in field_params["kernels"]:
                for len_scale, variance, nugget in itertools.product(
                    field_params["len_scales"],
                    field_params["variances"],
                    field_params["nuggets"]
                ):
                    for nu, alpha, beta in itertools.product(
                        field_params["nu"], field_params["alpha"], field_params["beta"]
                    ):
                        # === Directory structure ===
                        #subfolder = os.path.join(
                        #    out_dir,
                        #    f"seed_{seed}_grid{grid_size}_{kernel_name}_ls{len_scale}_var{variance}_nug{nugget}"
                        #)
                        #make_dir(subfolder)

                        subfolder = os.path.join(
                        out_dir,
                        f"master{master_seed}",
                        f"seed_{seed}_grid{grid_size}_{kernel_name}_ls{len_scale}_var{variance}_nug{nugget}")
                        make_dir(subfolder)

                        # === Generate SRFs ===
                        actual_srf, forecast_srf = generate_grf(
                            kernel_name,
                            dim=2,
                            var=variance,
                            len_scale=len_scale,
                            nugget=nugget,
                            nu=nu,
                            alpha=alpha,
                            beta=beta,
                            period=period,
                            seed=seed
                        )

                        # === Generate fields ===
                        actual_field   = actual_srf.structured(grid)
                        forecast_field = forecast_srf.structured(grid)

                        # === Normalize ===
                        actual_field   = normalize_field(actual_field)
                        forecast_field = normalize_field(forecast_field)

                        # === Ensure subfolder exists before saving ===
                        os.makedirs(subfolder, exist_ok=True)

                        # === Save outputs safely ===
                        np.save(os.path.join(subfolder, "actual.npy"), actual_field)
                        np.save(os.path.join(subfolder, "forecast.npy"), forecast_field)

                        # === Export CSVs for robust model and D* Lite compatibility ===
                        nodes_df = build_nodes(grid_size)


                        # Build 4-connected periodic graph using SRF data
                        edges_df = build_edges_4conn_nonperiodic(
                            grid_size=grid_size,
                            nominal_field=actual_field,                         # actual = nominal baseline
                            deviation_field=(forecast_field - actual_field)     # forecast = nominal + deviation
                        )

                        # Prepare DataFrames with required columns and order
                        edges_df = edges_df[["edge_id", "u", "v", "nominal", "deviation", "cost"]]
                        nodes_df = nodes_df[["node_id", "x", "y"]]

                        # Save nodes and edges csv files
                        nodes_df.to_csv(os.path.join(subfolder, "nodes.csv"), index=False)
                        edges_df.to_csv(os.path.join(subfolder, "edges.csv"), index=False)

                        print(f"Exported: 'nodes.csv','edges.csv' to {subfolder}")

                        if general_params["visualize"]:
                            save_plot(actual_field, f"{kernel_name}; actual SRF \n Cost = nominal", os.path.join(subfolder, "actual.png"))
                            save_plot(forecast_field, f"{kernel_name}; forecast SRF \n Cost = nominal + {nugget} nugget", os.path.join(subfolder, "forecast.png"))

                        # === Metadata ===
                        meta = {
                            "timestamp" : datetime.now().isoformat(),
                            "kernel"    : kernel_name,
                            "seed"      : seed,
                            "master_seed": master_seed,
                            "sub_seed": seed,
                            "grid_size" : grid_size,
                            "len_scale" : len_scale,
                            "variance"  : variance,
                            "nugget"    : nugget,
                            "nu"        : nu,
                            "alpha"     : alpha,
                            "beta"      : beta,
                            "period"    : period,
                        }
                        with open(os.path.join(subfolder, "metadata.json"), "w") as f:
                            json.dump(meta, f, indent=4)

                        print(f"Kernel: {kernel_name}, Grid size: {grid_size}, Subseed: {seed}")

    print("\nAll scenarios completed successfully!")

<h3 style="color:#2E86C1; font-family: 'Helvetica';">(5) Main</h2>

<p style="font-size: 16px; line-height: 1.5; color:#333;">
The data preprocessing part:</p>

In [589]:
if __name__ == "__main__":
    DataPostProcessingSRF("parameters.json")

→ Running 1 total scenarios from 1 master seeds: [999]
Exported: 'nodes.csv','edges.csv' to ../Data/master999/seed_30145_grid100_Stable_ls30.0_var1.0_nug5.0
Kernel: Stable, Grid size: 100, Subseed: 30145

All scenarios completed successfully!
